# **Proyecto: Generador de Letras de Taylor Swift con LSTM**

En este proyecto, exploraremos la emocionante intersección entre la música y la inteligencia artificial para crear un generador de letras de Taylor Swift. Utilizaremos una técnica de aprendizaje profundo conocida como Long Short Term Memory (LSTM) Recurrent Neural Networks.

Taylor Swift, una famosa cantante y compositora, es conocida por sus letras emotivas y cautivadoras. Con la ayuda de las LSTM, buscaremos capturar su estilo lírico distintivo y generar nuevas letras que mantengan la esencia de su música.

Las LSTM son una variante de las redes neuronales recurrentes diseñadas para modelar secuencias de datos. Con su capacidad para capturar dependencias a largo plazo en las secuencias, las LSTM son ideales para generar letras de canciones coherentes y auténticas.

En este proyecto, entrenaremos un modelo LSTM utilizando un corpus de letras de Taylor Swift. Alimentaremos el modelo con estas letras para que aprenda los patrones y estructuras líricas característicos de su música. Una vez entrenado, el modelo será capaz de generar nuevas letras de Taylor Swift de manera automática.



Importamos las librerias necesarias y la base de datos

In [ ]:
import numpy as np
import pandas as pd
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [ ]:
path="/content/drive/MyDrive/taylor_swift_lyrics.csv"
dataset=  pd.read_csv(path, encoding = "latin1")

In [ ]:
dataset.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [ ]:
dataset.describe()

,track_n,line,year
count,4862.000000,4862.000000,4862.000000
mean,8.216989,28.426573,2011.882764
std,4.696379,18.343649,3.571447
min,1.000000,1.000000,2006.000000
25%,4.000000,13.000000,2010.000000
50%,8.000000,26.000000,2012.000000
75%,12.000000,41.000000,2014.000000
max,19.000000,101.000000,2017.000000


Ahora concatenamos las líneas de cada canción para obtener cada canción por separado en una cadena.

In [ ]:
def processFirstLine(lyrics, songID, songName, row):
    lyrics.append(row['lyric'] + '\n')
    songID.append( row['year']*100+ row['track_n'])
    songName.append(row['track_title'])
    return lyrics,songID,songName

In [ ]:
# Definir listas vacías para las letras, el ID de la canción y el nombre de la canción 
lyrics = []
songID = []
songName = []

# songNumber indica el número de la canción en el conjunto de datos
songNumber = 1

# i indica el número de la canción
i = 0
isFirstLine = True

# Iterar a través de cada línea de letras y unirlas para cada canción de manera independiente 
for index,row in dataset.iterrows():
    if(songNumber == row['track_n']):
        if (isFirstLine):
            lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
            isFirstLine = False
        else :
            #si estamos en la misma canción, se unen las lineas de las canciones    
            lyrics[i] +=  row['lyric'] + '\n'
    #si ya estan todas las partes de la canción seguir con la otra:    
    else :
        lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
        songNumber = row['track_n']
        i+=1

In [ ]:
# Definimos un nuevo marco de datos de pandas para guardar el ID de la canción, el nombre de la canción y las letras para usarlos más tarde
lyrics_data = pd.DataFrame({'songID':songID, 'songName':songName, 'lyrics':lyrics })


Guardamos las canciones completas en formato .txt

In [ ]:
with open('lyricsText.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in lyrics:
        filehandle.write('%s\n' % listitem)

## Procesamiento de datos

Convertimos todo el contenido en minúsculas 

In [ ]:
textFileName = 'lyricsText.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()
raw_text = raw_text.lower()


### Mapping

Creamos dos diccionarios, uno para convertir caracteres a enteros, el otro para convertir enteros de nuevo a caracteres:

In [ ]:
#chars a ints :
chars = sorted(list(set(raw_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((i, c) for c, i in enumerate(chars))

In [ ]:
# número de caracteres y vocabulario en nuestro texto:
n_chars = len(raw_text)
n_vocab = len(chars)

In [ ]:
print('Total Characters : ' , n_chars) #caracteres en lyricsText.txt
print('Total Vocab : ', n_vocab) #caracteres únicos

Total Characters :  173698
Total Vocab :  58


## Realización de muestras y etiquetas para alimentar la LSTM RNN 

In [ ]:
# process the dataset:
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    # Input Sequeance(will be used as samples)
    seq_in  = raw_text[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = raw_text[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)

Total Patterns :  173598


Prepararemos las muestras y las etiquetas para que estén listas para entrenar nuestro modelo.
* Reformar las muestras
* Normalizarlos
* Codificar las salidas

In [ ]:
# Remodelar X para ser adecuado para entrar en LSTM RNN:
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizamos los datos :
X = X/ float(n_vocab)
#Codificamos salidas:
y = np_utils.to_categorical(data_y)

# Construimos el modelo

Comenzaremos determinando cuántas capas tendrá nuestro modelo y cuántos nodos tendrá cada capa.

In [ ]:
LSTM_layer_num = 4 #capas
layer_size = [256,256,256,256] #nodos

Definimos nuestro modelo secuencial

In [ ]:
model = Sequential()

Hacemos uso de CuDNNLSTM que es 15 veces más rápido LSTM.

In [ ]:
model.add(CuDNNLSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))

In [ ]:
for i in range(1,LSTM_layer_num) :
    model.add(CuDNNLSTM(layer_size[i], return_sequences=True))

In [ ]:
model.add(Flatten())

Agregamos una capa de salida y definimos su función de activación y compilamos el modelo con los parametros de pédida y optimización

In [ ]:
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

Resumimos nuestro nuevo modelo

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 100, 256)          265216    
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 100, 256)          526336    
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 100, 256)          526336    
                                                                 
 cu_dnnlstm_3 (CuDNNLSTM)    (None, 100, 256)          526336    
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 58)                1484858   
                                                                 
 activation (Activation)     (None, 58)                0

Con este output vemos que todos nuestros parámetros son entrenables


```
Total params: 3,329,082
Trainable params: 3,329,082
Non-trainable params: 0
```



Realizamos un Callback

¿Qué es un callback?
es una función que se llama después de cierto tiempo.

Para este caso se llamará a un punto de control para guardar los resultados del modelo

In [ ]:
#Hacemos el punto de control
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

# Entrenando al modelo

In [ ]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])


Train on 138878 samples, validate on 34720 samples
Epoch 1/30
138878/138878 [==============================] - 186s 1ms/step - loss: 2.8300 - val_loss: 2.7864

Epoch 00001: loss improved from 3.00537 to 2.82996, saving model to Weights-LSTM-improvement-001-2.82996-bigger.hdf5
Epoch 2/30
138878/138878 [==============================] - 186s 1ms/step - loss: 2.6424 - val_loss: 2.6723

Epoch 00002: loss improved from 2.82996 to 2.64236, saving model to Weights-LSTM-improvement-002-2.64236-bigger.hdf5
Epoch 3/30
138878/138878 [==============================] - 186s 1ms/step - loss: 2.3721 - val_loss: 2.5978

Epoch 00003: loss improved from 2.64236 to 2.37208, saving model to Weights-LSTM-improvement-003-2.37208-bigger.hdf5
Epoch 4/30
138878/138878 [==============================] - 186s 1ms/step - loss: 1.9650 - val_loss: 2.6343

Epoch 00004: loss improved from 2.37208 to 1.96500, saving model to Weights-LSTM-improvement-004-1.96500-bigger.hdf5
Epoch 5/30
138878/138878 [===================

In [ ]:
# Pesamos la exactitud del modelo :
wights_file = './models/Weights-LSTM-improvement-004-2.49538-bigger.hdf5' # weights file path
model.load_weights(wights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [ ]:
# set a random seed :
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
print('Seed : ')
print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")

# How many characters you want to generate
generated_characters = 300

# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = int_chars[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')


Seed : 
"  once, i've been waiting, waiting
ooh whoa, ooh whoa
and all at once, you are the one, i have been w "

eu h mool shoea
a eir, bo ly lean on the sast
is tigm's the noen uo doy, fo shey stant tas you fot you srart aoo't you tein so my liost
i spaye 
somethppel' cua
iy yas tn mu, io' me
ohehip in the uorlirs tiines ho a ban't teit dven aester, tee tame
mnweiny you'd be pe k bet thing
oe eowt the light i
Done
